In [ ]:
!pip install transformers datasets peft accelerate torch

In [ ]:
import torch
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, TrainingArguments, Trainer, DataCollatorForSeq2Seq
from peft import LoraConfig, get_peft_model, TaskType, prepare_model_for_kbit_training

In [ ]:
model_checkpoint = "google/flan-t5-small"

In [ ]:
# Load tokenizer and model
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

tokenizer_config.json: 0.00B [00:00, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

In [ ]:
!pip install bitsandbytes

In [ ]:
# Load the base model. We use device_map="auto" to leverage accelerate for placing layers across devices.
# We also load in 8-bit for further memory saving, compatible with LoRA.
# Note: 8-bit loading is optional but useful for larger models.
# If not using 8-bit, remove load_in_8bit and prepare_model_for_kbit_training
model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint, device_map="auto")

model.safetensors:   0%|          | 0.00/308M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [ ]:
model = prepare_model_for_kbit_training(model)


In [ ]:
# Load the dataset
dataset_name = "spencer/samsum_reformat"
dataset = load_dataset(dataset_name, split="train[:1%]") # Using only 1% for demo
dataset = dataset.train_test_split(test_size=0.1) # Create train/test splits

print(f"Train dataset size: {len(dataset['train'])}")
print(f"Test dataset size: {len(dataset['test'])}")
# Example: Train dataset size: 132
# Example: Test dataset size: 15

dataset_infos.json: 0.00B [00:00, ?B/s]

data/train-00000-of-00001.parquet:   0%|          | 0.00/16.6M [00:00<?, ?B/s]

data/validation-00000-of-00001.parquet:   0%|          | 0.00/923k [00:00<?, ?B/s]

data/test-00000-of-00001.parquet:   0%|          | 0.00/968k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/14732 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/818 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/819 [00:00<?, ? examples/s]

Train dataset size: 132
Test dataset size: 15


In [ ]:
dataset.data

{'train': MemoryMappedTable
 id: string
 dialogue: string
 summary: string
 sentences: list<item: string>
   child 0, item: string
 sentence_id: list<item: string>
   child 0, item: string
 dialog_id: string
 ----
 id: [["13818513","13728867","13681000","13730747","13728094",...,"13829461","13828209","13864466","13862429","13819578"]]
 dialogue: [["Amanda: I baked  cookies. Do you want some?
 Jerry: Sure!
 Amanda: I'll bring you tomorrow :-)","Olivia: Who are you voting for in this election? 
 Oliver: Liberals as always.
 Olivia: Me too!!
 Oliver: Great","Tim: Hi, what's up?
 Kim: Bad mood tbh, I was going to do lots of stuff but ended up procrastinating
 Tim: What did you plan on doing?
 Kim: Oh you know, uni stuff and unfucking my room
 Kim: Maybe tomorrow I'll move my ass and do everything
 Kim: We were going to defrost a fridge so instead of shopping I'll eat some defrosted veggies
 Tim: For doing stuff I recommend Pomodoro technique where u use breaks for doing chores
 Tim: It rea

In [ ]:
# Preprocessing function
max_input_length = 512
max_target_length = 128

In [ ]:
def preprocess_function(examples):
    # Add prefix for T5 models
    inputs = ["summarize: " + doc for doc in examples["dialogue"]]
    model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True, padding="max_length")

    # Setup the tokenizer for targets
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(examples["summary"], max_length=max_target_length, truncation=True, padding="max_length")

    model_inputs["labels"] = labels["input_ids"]
    # Replace tokenizer.pad_token_id in the labels by -100 to ignore padding in the loss calculation
    model_inputs["labels"] = [
        [(l if l != tokenizer.pad_token_id else -100) for l in label] for label in model_inputs["labels"]
    ]
    return model_inputs

In [ ]:
# Apply preprocessing
tokenized_datasets = dataset.map(preprocess_function, batched=True)

Map:   0%|          | 0/132 [00:00<?, ? examples/s]

/usr/local/lib/python3.12/dist-packages/transformers/tokenization_utils_base.py:4118: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/15 [00:00<?, ? examples/s]

In [ ]:
# Remove columns not needed for training
tokenized_datasets = tokenized_datasets.remove_columns(["id", "dialogue", "summary"])

print(f"Columns in tokenized dataset: {tokenized_datasets['train'].column_names}")
# Example: Columns in tokenized dataset: ['input_ids', 'attention_mask', 'labels']

Columns in tokenized dataset: ['sentences', 'sentence_id', 'dialog_id', 'input_ids', 'attention_mask', 'labels']


In [ ]:
tokenized_datasets = tokenized_datasets.remove_columns(["sentences", "sentence_id", "dialog_id"])

In [ ]:
print(f"Columns in tokenized dataset: {tokenized_datasets['train'].column_names}")

Columns in tokenized dataset: ['input_ids', 'attention_mask', 'labels']


In [ ]:
# Create data collator
data_collator = DataCollatorForSeq2Seq(
    tokenizer,
    model=model,
    label_pad_token_id=-100, # Important: ensure labels are padded correctly
    pad_to_multiple_of=8 # Optional: optimizes hardware usage
)

In [ ]:
# Define LoRA configuration
lora_config = LoraConfig(
    r=16, # Rank of the update matrices
    lora_alpha=32, # Scaling factor
    target_modules=["q", "v"], # Apply LoRA to query and value projections
    lora_dropout=0.05, # Dropout probability
    bias="none", # Do not train biases
    task_type=TaskType.SEQ_2_SEQ_LM # Task type for sequence-to-sequence models
)

In [ ]:
# Get the PEFT model
peft_model = get_peft_model(model, lora_config)

# Print the number of trainable parameters
peft_model.print_trainable_parameters()
# Example output: trainable params: 884,736 || all params: 77,822,464 || trainable%: 1.13685..

trainable params: 688,128 || all params: 77,649,280 || trainable%: 0.8862


In [ ]:
# Define Training Arguments
output_dir = "flan-t5-small-samsum-lora"
training_args = TrainingArguments(
    output_dir=output_dir,
    auto_find_batch_size=True, # Automatically find a suitable batch size
    learning_rate=1e-3, # Higher learning rate typical for LoRA
    num_train_epochs=3, # Number of training epochs
    logging_strategy="epoch", # Log metrics every epoch
    save_strategy="epoch", # Save checkpoint every epoch
    # evaluation_strategy="epoch", # Evaluate every epoch if eval data is available
    report_to="none", # Disable reporting to wandb/tensorboard for this example
    # Use fp16 for faster training if supported
    # fp16=torch.cuda.is_available(),
)

In [ ]:
# Create Trainer instance
trainer = Trainer(
    model=peft_model, # Pass the PEFT model
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"], # Optional: Pass eval dataset
    data_collator=data_collator,
    tokenizer=tokenizer,
)

/tmp/ipython-input-450767527.py:2: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
The model is already on multiple devices. Skipping the move to device specified in `args`.


In [ ]:
# Set LoRA layers to trainable explicitly (sometimes needed)
peft_model.config.use_cache = False # Disable caching for training

In [ ]:
# Start training
print("Starting LoRA training...")
trainer.train()
print("Training finished.")

Starting LoRA training...


/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:668: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Step,Training Loss
17,2.671300
34,2.223800


/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:668: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:668: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Step,Training Loss
17,2.671300
34,2.223800
51,2.156100


Training finished.


In [ ]:
# Define path to save the adapter
adapter_path = f"{output_dir}/final_adapter"

# Save the adapter weights
peft_model.save_pretrained(adapter_path)
tokenizer.save_pretrained(adapter_path) # Save tokenizer alongside adapter

print(f"LoRA adapter saved to: {adapter_path}")

# You can check the size of the saved adapter - it should be relatively small (MBs).
# For example, using: !ls -lh {adapter_path}

LoRA adapter saved to: flan-t5-small-samsum-lora/final_adapter


In [ ]:
!ls -lh /content/flan-t5-small-samsum-lora

total 16K
drwxr-xr-x 2 root root 4.0K Dec  1 15:43 checkpoint-17
drwxr-xr-x 2 root root 4.0K Dec  1 15:49 checkpoint-34
drwxr-xr-x 2 root root 4.0K Dec  1 15:54 checkpoint-51
drwxr-xr-x 2 root root 4.0K Dec  1 16:06 final_adapter


In [ ]:
from peft import PeftModel, PeftConfig

# Load the base model again (if not already in memory)
base_model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint, torch_dtype=torch.float16, device_map="auto")
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

`torch_dtype` is deprecated! Use `dtype` instead!


In [ ]:
# Load the PEFT model with the saved adapter
lora_model = PeftModel.from_pretrained(base_model, adapter_path)
lora_model = lora_model.to("cuda" if torch.cuda.is_available() else "cpu") # Ensure model is on correct device
lora_model.eval() # Set model to evaluation mode

PeftModelForSeq2SeqLM(
  (base_model): LoraModel(
    (model): T5ForConditionalGeneration(
      (shared): Embedding(32128, 512)
      (encoder): T5Stack(
        (embed_tokens): Embedding(32128, 512)
        (block): ModuleList(
          (0): T5Block(
            (layer): ModuleList(
              (0): T5LayerSelfAttention(
                (SelfAttention): T5Attention(
                  (q): lora.Linear(
                    (base_layer): Linear(in_features=512, out_features=384, bias=False)
                    (lora_dropout): ModuleDict(
                      (default): Dropout(p=0.05, inplace=False)
                    )
                    (lora_A): ModuleDict(
                      (default): Linear(in_features=512, out_features=16, bias=False)
                    )
                    (lora_B): ModuleDict(
                      (default): Linear(in_features=16, out_features=384, bias=False)
                    )
                    (lora_embedding_A): ParameterDict()
            

In [ ]:
# Prepare a sample input from the test set (or any new dialogue)
sample_idx = 5
dialogue = dataset['test'][sample_idx]['dialogue']
reference_summary = dataset['test'][sample_idx]['summary']

In [ ]:
input_text = "summarize: " + dialogue
input_ids = tokenizer(input_text, return_tensors="pt").input_ids.to(lora_model.device)

In [ ]:
print("Dialogue:")
print(dialogue)
print("\nReference Summary:")
print(reference_summary)

Dialogue:
Julia: Adam, are you coming today?
Julia: Adam, you are already an hour late, let me know asap
Kate: He texted me before that he isn't feeling very well
Julia: Thanks
Adam: I had an appointment, sorry, but I have a stomach flu

Reference Summary:
Adam has a stomach flu. 


In [ ]:
# Generate summary using the LoRA model
with torch.no_grad():
    outputs = lora_model.generate(input_ids=input_ids, max_new_tokens=100, do_sample=True, top_p=0.9)
generated_summary = tokenizer.decode(outputs[0], skip_special_tokens=True)

print("\nGenerated Summary (LoRA):")
print(generated_summary)


Generated Summary (LoRA):
A has been called to an appointment at an earlier time.
